In [1]:
import pulp

In [2]:
import numpy as np
import pandas as pd
import math

In [3]:
members = ["Dan","Steven","Jacinta"]

In [4]:
days = range(1,9)

In [5]:
# defining the variables
x = pulp.LpVariable.dicts("x",(members,days),0,1,pulp.LpInteger)

In [6]:
print(x)

{'Dan': {1: x_Dan_1, 2: x_Dan_2, 3: x_Dan_3, 4: x_Dan_4, 5: x_Dan_5, 6: x_Dan_6, 7: x_Dan_7, 8: x_Dan_8}, 'Steven': {1: x_Steven_1, 2: x_Steven_2, 3: x_Steven_3, 4: x_Steven_4, 5: x_Steven_5, 6: x_Steven_6, 7: x_Steven_7, 8: x_Steven_8}, 'Jacinta': {1: x_Jacinta_1, 2: x_Jacinta_2, 3: x_Jacinta_3, 4: x_Jacinta_4, 5: x_Jacinta_5, 6: x_Jacinta_6, 7: x_Jacinta_7, 8: x_Jacinta_8}}


In [7]:
# objective
model = pulp.LpProblem("Warmup", pulp.LpMinimize)

# objective function : minimize the number of warmups each member has to do
for m in members:
    model += pulp.lpSum(x[m][d] for d in days)

C:\Users\luket\AppData\Local\Programs\Python\Python310\lib\site-packages\pulp\pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [8]:
# every member has run to the warmup at least once
for m in members:
    model += pulp.lpSum(x[m][d] for d in days) >= 1

In [9]:
# nobody should have to run the warmup more than total/members rounded up (8/3 = 3)
for m in members:
    model += pulp.lpSum(x[m][d] for d in days) <= math.ceil(len(days)/len(members))

In [10]:
# every warmup must have a leader
for d in days:
    model += pulp.lpSum(x[m][d] for m in members) == 1

In [11]:
# number of sessions between running warmups : 1
for m in members:
    for d in days:
        if d>1:
            model += x[m][d] + x[m][d-1] <= 1

In [12]:
# number of sessions between running warmups : 2
#for m in members:
#    for d in days:
#        if d>2:
#            model += x[m][d] + x[m][d-1] + x[m][d-2] <= 1
            
# a bit of a shitty way to enforce gaps
# improvements??

In [13]:
tuesday_generator = [d for d in days if d%2==1]
thursday_generator = [d for d in days if d%2==0]

In [14]:
# Dan is only available on Tuesdays (odd numbered sessions)
dan_generator = [m for m in members if m=="Dan"]

for m in dan_generator:
    for d in thursday_generator:
        model += x[m][d] == 0

## Solving the Model

In [15]:
# Solve ILP problem and post-processing to get the summary
model.solve()

1

In [16]:
# model

In [17]:
for vi in model.variables():
    if vi.varValue == 1:
        print(f"{vi.name} = {vi.varValue:.0f}")

x_Dan_3 = 1
x_Dan_5 = 1
x_Dan_7 = 1
x_Jacinta_1 = 1
x_Jacinta_4 = 1
x_Steven_2 = 1
x_Steven_6 = 1
x_Steven_8 = 1


## Converting to a dataframe for a clean output

In [18]:
myperson = []
myday = []
value = []

for vi in model.variables():
    person = vi.name.split("_")[1]
    day = vi.name.split("_")[2]
    
    myperson.append(person)
    myday.append(day)
    value.append(vi.varValue)
    
data = {'Day':myday, 'Person':myperson, 'Value':value} 

df = pd.DataFrame(data)

In [19]:
df.sort_values(by='Day', ascending=True)

,Day,Person,Value
0,1,Dan,0.0
16,1,Steven,0.0
8,1,Jacinta,1.0
1,2,Dan,0.0
17,2,Steven,1.0
9,2,Jacinta,0.0
2,3,Dan,1.0
18,3,Steven,0.0
10,3,Jacinta,0.0
19,4,Steven,0.0


In [20]:
# df.to_csv('role_optimazation.csv', index = False)